In [1]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import numpy as np
import pandas_datareader as pdr
from io import StringIO
import yfinance as yf
import os

import time
import requests
import datetime as dt
import openpyxl
from openpyxl import workbook as wb
from openpyxl import load_workbook

from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
#read portfolio tickers from most recent GIF Historical Closes file

today = dt.datetime.today().strftime("%m.%d.%Y")
ClosePath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_historical_closes11.11.2022.xlsx'

tickerdf=pd.read_excel(ClosePath,sheet_name='GIF_YTD_Perf',index_col=0)

#dataframe should just be ticker and current value.
tickerdf=tickerdf.iloc[:,[0]]
tickerdf=tickerdf.sort_values('Ticker')
tickerdf.insert(0,"Company Name","")
#tickerdf

In [3]:
#Read in relevant details from cost basis and close prices files
BasisPath=f'C:/Users/Will_Boisseau/OneDrive - Georgetown University/Graduate Investment Fund (GIF)/All Things Investing/Performance & Research/Performance/Brokerage Data_for all club members/GIF_CostBasis_asof11.11.2022.xlsx'

#get share amounts for both brokerages
basisdf=pd.read_excel(BasisPath,sheet_name='cost_raw')
basiscols=[2,4,5]
basisdf=basisdf[basisdf.columns[basiscols]]
basisdf['Total Shares']=basisdf['MS_Quantity']+basisdf['BNY_Quantity']
basisdf=basisdf.sort_values('Ticker')
basisdf=basisdf.drop(columns=["Ticker"])
#0 decimals for share columns
basisdf.loc[:, "MS_Quantity"] = basisdf["MS_Quantity"].map('{:.0f}'.format)
basisdf.loc[:, "BNY_Quantity"] = basisdf["BNY_Quantity"].map('{:.0f}'.format)
basisdf.loc[:, "Total Shares"] = basisdf["Total Shares"].map('{:.0f}'.format)

#get purchase date - still need to format dates correctly
datedf=pd.read_excel(BasisPath,sheet_name='GIF_attributes-cost')
datecols=[2,9]
datedf=datedf[datedf.columns[datecols]]
datedf=datedf.sort_values('Ticker')
datedf=datedf.drop(columns=["Ticker"])
#format the dates correctly
datedf['Entry_date']=pd.to_datetime(datedf['Entry_date'])

#combine and reorganize the dataframes
combdf=pd.concat([tickerdf,basisdf,datedf],axis=1)
combdf.reset_index(inplace=True,drop=True)

#put cash at the top row
target_row=combdf[combdf['Ticker']=="Cash_Balance"].index[0]
idx = [target_row] + [i for i in range(len(combdf)) if i != target_row]
combdf=combdf.iloc[idx]
combdf.reset_index(inplace=True,drop=True)
combdf[['Total Shares']]=combdf[['Total Shares']].fillna("input value")
combdf



,Company Name,Ticker,MS_Quantity,BNY_Quantity,Total Shares,Entry_date
0,,Cash_Balance,NaN,NaN,input value,NaT
1,,AAPL,65,180,245,2007-11-15
2,,AMT,0,57,57,2021-02-01
3,,AON,127,0,127,2021-04-19
4,,BEP,0,512,512,2021-02-16
5,,BRK-B,30,50,80,2013-01-25
6,,BXMT,0,534,534,2022-05-02
7,,CAT,45,70,115,2010-02-18
8,,CI,124,0,124,2021-12-01
9,,DAL,0,950,950,2021-10-06


In [4]:
#Source data from Yahoo API. This step isn't actually necessary - can use data from Bing in Excel

##first create ticker list from the dataframe
#ticker_list = tickerdf['Ticker'].tolist()

##create an empty list
#price_list=[]
#beta_list=[]

##use a loop to get latest price for each holding into a list, then convert to dataframe
#for tik in ticker_list:
    #ticker=yf.Ticker(tik).info
    ##assign values for each attribute
   # price=ticker['regularMarketPrice']
    #beta=ticker['beta']
   # #add them to the respective list
    #price_list.append(price)
    #beta_list.append(beta)
    
##clean up the dataframes
#pricedf=pd.DataFrame([price_list])
#pricedf=pricedf.transpose()
#pricedf.rename(columns={0:'Latest Price'},inplace=True)

#betadf=pd.DataFrame([beta_list])
#betadf=betadf.transpose()
#betadf.rename(columns={0:'Beta'},inplace=True)



In [5]:
#save data, without overwriting, to 'Portfolio Trader' excel file. This adds columns, but doesn't overwrite the ones I want

TraderPath=r'C:\Users\Will_Boisseau\OneDrive - Georgetown University\GIF files\Portfolio Trader.xlsx'

writer = pd.ExcelWriter(TraderPath, engine='openpyxl', mode='a',if_sheet_exists='overlay')

# try to open an existing workbook
writer.book = load_workbook(TraderPath)   
combdf.to_excel(writer,'Combined Portfolio', startrow=0,columns=['Company Name','Ticker','MS_Quantity','BNY_Quantity','Total Shares','Entry_date'])
writer.save()
combdf2=pd.read_excel(TraderPath,sheet_name='Combined Portfolio',index_col=0)
combdf2
#combdf


C:\Users\WILL_B~1\AppData\Local\Temp/ipykernel_28344/2581634796.py:8: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = load_workbook(TraderPath)
C:\Users\WILL_B~1\AppData\Local\Temp/ipykernel_28344/2581634796.py:10: FutureWarning: save is not part of the public API, usage can give unexpected results and will be removed in a future version
  writer.save()


,Company Name,Ticker,MS_Quantity,BNY_Quantity,Total Shares,Entry_date,Latest Price,Total Value,$ Unrealized Gain,% Unrealized Gain,Weighting,Industry,GICS Sector,% change,$ change
Index,,,,,,,,,,,,,,,
0,NaN,Cash_Balance,NaN,NaN,input value,NaT,43.195,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AAPL,65.0,180.0,245,2007-11-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,AMT,0.0,57.0,57,2021-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,AON,127.0,0.0,127,2021-04-19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,BEP,0.0,512.0,512,2021-02-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,BRK-B,30.0,50.0,80,2013-01-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,BXMT,0.0,534.0,534,2022-05-02,NaN,NaN,NaN,bitch,NaN,NaN,NaN,NaN,NaN
7,NaN,CAT,45.0,70.0,115,2010-02-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,CI,124.0,0.0,124,2021-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
